# Minimização com restrições lineares de igualdade

$$
\min \quad f(x) \\
\mbox{suj. a } \quad Ax = b
$$

## Base para o núcleo de $A$

$$ QR = A^T $$

In [2]:
A = rand(3, 5)

3x5 Array{Float64,2}:
 0.786041  0.518244  0.635686  0.411794  0.975438
 0.100398  0.871222  0.797831  0.326629  0.764526
 0.935218  0.925968  0.243322  0.950776  0.575103

In [5]:
Q, R = qr(A', thin=false)

(
5x5 Array{Float64,2}:
 -0.506167   0.693999     0.198824  -0.288985     -0.372984  
 -0.33372   -0.607197     0.481964  -0.536314     -0.00445586
 -0.409346  -0.386597    -0.413945   0.325702     -0.636825  
 -0.265172   0.00113121   0.62362    0.72303       0.134195  
 -0.628128   0.014817    -0.409789   0.000320581   0.661293  ,

3x3 Array{Float64,2}:
 -1.55293  -1.23498   -1.49535   
  0.0      -0.756068   0.00232437
  0.0       0.0        0.888758  )

In [10]:
Rtil = [R; zeros(2,3)]

5x3 Array{Float64,2}:
 -1.55293  -1.23498   -1.49535   
  0.0      -0.756068   0.00232437
  0.0       0.0        0.888758  
  0.0       0.0        0.0       
  0.0       0.0        0.0       

In [11]:
Rtil - Q'*A'

5x3 Array{Float64,2}:
 -2.22045e-16   0.0           2.22045e-16
  8.22974e-18  -1.11022e-16  -2.38091e-16
 -6.95113e-17  -7.16247e-17  -2.22045e-16
  1.57913e-16   8.9784e-17    3.22812e-16
 -4.0278e-16   -1.69304e-16  -1.70538e-16

In [14]:
A*Q

3x5 Array{Float64,2}:
 -1.55293  -8.22974e-18  6.95113e-17  -1.57913e-16  4.0278e-16 
 -1.23498  -0.756068     7.16247e-17  -8.9784e-17   1.69304e-16
 -1.49535   0.00232437   0.888758     -3.22812e-16  1.70538e-16

In [17]:
Z = Q[:,4:5]

5x2 Array{Float64,2}:
 -0.288985     -0.372984  
 -0.536314     -0.00445586
  0.325702     -0.636825  
  0.72303       0.134195  
  0.000320581   0.661293  

In [1]:
function newton_method(f, x0; tol = 1e-5, max_iter = 1000, max_time = 60)
    exit_flag = 0
    ∇f(x) = ForwardDiff.gradient(f, x)
    H(x) = ForwardDiff.hessian(f, x)
    
    x = copy(x0) # Cópia de x0
    iter = 0
    start_time = time()
    elapsed_time = 0.0
    fx = f(x)
    ∇fx = ∇f(x)
    B = H(x)
    while norm(∇fx) > tol
        d = -B\∇fx
        ∇fx_dot_d = dot(∇fx, d)
        if ∇fx_dot_d > -1e-6 * norm(∇fx) * norm(d)
            d = -∇fx
            ∇fx_dot_d = dot(∇fx, d)
        elseif norm(d) < 1e-6 * norm(∇fx)
            d = -∇fx
            ∇fx_dot_d = dot(∇fx, d)
        end
        t = 1.0
        while f(x + t*d) > fx + 0.5*t*∇fx_dot_d
            t = t*0.9
        end
        x = x + t*d
        fx = f(x)
        ∇fx = ∇f(x)
        B = H(x)
        iter = iter + 1
        if iter >= max_iter
            exit_flag = 1
            break
        end
        elapsed_time = time() - start_time
        if elapsed_time >= max_time
            exit_flag = 2
            break
        end
    end
    return x, fx, ∇fx, exit_flag, iter, elapsed_time # Precisamos retornar o ponto encontrado
end

newton_method (generic function with 1 method)

In [3]:
using ForwardDiff
f(x) = (1-x[1])^2 + 100*(x[2]-x[1]^2)^2
newton_method(f, zeros(2))

([0.9999999999986602,0.999999999997286],1.913562380330407e-24,[1.1087131213098218e-11,-6.8833827526759706e-12],0,14,1.516280174255371)

In [50]:
n = 10
f(x) = dot(x,x)
A = ones(1,n)
b = [1.0]
xbarra = [1.0; zeros(n-1)]

10-element Array{Float64,1}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [51]:
A*xbarra - b
Q,R = qr(A', thin=false)
Z = Q[:,2:end]
ϕ(u) = f(xbarra + Z*u)

ϕ (generic function with 1 method)

In [52]:
u = rand(n-1)
ϕ(u)

0.7610585249391503

In [53]:
f(xbarra + Z*u)

0.7610585249391503

In [54]:
using ForwardDiff
u, ϕu, ngu, ef, iter, dt = newton_method(ϕ, zeros(n-1))

-1.7999999999999998
-0.017999999999999943
-0.0001800000000000008
-1.7999999999994466e-6
-1.7999999999967828e-8
-1.8000000000603945e-10


([0.3162274497890719,0.3162274497890719,0.31622744978907197,0.31622744978907197,0.3162274497890719,0.3162274497890719,0.3162274497890719,0.3162274497890719,0.3162274497890719],0.10000000000090022,[-6.324555318938663e-7,-6.324555318938663e-7,-6.32455531713455e-7,-6.32455531782844e-7,-6.324555318938663e-7,-6.324555318799885e-7,-6.324555318834579e-7,-6.324555318869274e-7,-6.324555318903968e-7],0,6,0.05267000198364258)

In [55]:
u

9-element Array{Float64,1}:
 0.316227
 0.316227
 0.316227
 0.316227
 0.316227
 0.316227
 0.316227
 0.316227
 0.316227

In [56]:
sol = xbarra + Z*u

10-element Array{Float64,1}:
 0.100001 
 0.0999999
 0.0999999
 0.0999999
 0.0999999
 0.0999999
 0.0999999
 0.0999999
 0.0999999
 0.0999999

## Sistema das condições necessárias

In [65]:
L(x,λ) = f(x) + dot(λ, A*x-b)

L (generic function with 1 method)

In [66]:
x, fx, ngx, ef, iter, dt = newton_method(v->L(v[1:n],[v[end]]), zeros(n+1))

([-1.3848383662915744e230,-1.3848383662915744e230,-1.3848383662915744e230,-1.3848383662915744e230,-1.3848383662915744e230,-1.3848383662915744e230,-1.3848383662915744e230,-1.3848383662915744e230,-1.3848383662915744e230,-1.3848383662915744e230,2.769676732583149e230],NaN,[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.3848383662915744e231],1,1000,0.10395097732543945)

In [59]:
q(v) = L(v[1:n],v[end:end])

q (generic function with 1 method)

In [60]:
q(zeros(n+1))

0.0

In [61]:
∇q(x) = ForwardDiff.gradient(q, x)
H(x) = ForwardDiff.hessian(q, x)

H (generic function with 1 method)

In [67]:
d = - H(zeros(n+1))\ ∇q(zeros(n+1))

11-element Array{Float64,1}:
  0.1
  0.1
  0.1
  0.1
  0.1
  0.1
  0.1
  0.1
  0.1
  0.1
 -0.2

In [63]:
∇q(sol)

11-element Array{Float64,1}:
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
 -1.11022e-16

In [68]:
x = zeros(n+1)

11-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [69]:
x + d

11-element Array{Float64,1}:
  0.1
  0.1
  0.1
  0.1
  0.1
  0.1
  0.1
  0.1
  0.1
  0.1
 -0.2

In [78]:
q(x + d) > q(x) + 0.5 * dot(∇q(x), d)

true

In [79]:
# Passo de Newton não é aceito por Armijo

In [83]:
t = 0.9
while q(x + t*d) > q(x) + 0.5 * t * dot(∇q(x), d)
    println("q+ = $(q(x + t*d))")
    println("qdot = $(q(x) + 0.5 * t * dot(∇q(x), d))")
    t = t * 0.9
    if t < 0.1
        break
    end
end

q+ = 0.099
qdot = 0.09000000000000001
q+ = 0.09639000000000003
qdot = 0.08100000000000002
q+ = 0.09265590000000001
qdot = 0.0729
q+ = 0.08817327900000001
qdot = 0.06561000000000002
q+ = 0.08323015599000003
qdot = 0.05904900000000002
q+ = 0.07804524635190002
qdot = 0.05314410000000002
q+ = 0.07278258754503901
qdot = 0.047829690000000015
q+ = 0.0675632401114816
qdot = 0.04304672100000002
q+ = 0.06247463427030012
qdot = 0.03874204890000002
q+ = 0.05757802256094309
qdot = 0.03486784401000002
q+ = 0.05291441019616392
qdot = 0.03138105960900002
q+ = 0.048509262988512776
qdot = 0.02824295364810002
q+ = 0.04437623467735334
qdot = 0.025418658283290013
q+ = 0.04052010857964841
qdot = 0.022876792454961013
q+ = 0.0369391105914082
qdot = 0.02058911320946491
q+ = 0.03362671995674432
qdot = 0.018530201888518418
q+ = 0.030573079504896215
qdot = 0.016677181699666577
q+ = 0.027766087104905926
qdot = 0.015009463529699922
q+ = 0.025192233990319782
qdot = 0.013508517176729929
q+ = 0.02283724262397041
qdot 

0.09000000000000001